# Run Test Method

In [16]:
def runTests(testClass):
    loader = unittest.TestLoader()
    suite = loader.loadTestsFromTestCase(testClass)
    runner = unittest.TextTestRunner()
    runner.run(suite)

# Modelo

In [17]:
from __future__ import annotations
import ast, sys
from dataclasses import dataclass
from typing import List, Iterable, Tuple, Optional

@dataclass(frozen=True)
class CodeWarning:
    message: str
    lineno: int
    col: int
    end_lineno: Optional[int] = None
    end_col: Optional[int] = None

class DetectionRule:
    name: str = "Generic Detector"
    def search(self, src: str) -> List[CodeWarning]:
        raise NotImplementedError

class Patcher:
    name: str = "Generic Patcher"
    def patch(self, tree: ast.Node) -> str:
        raise NotImplementedError

class DCCLinter:
    def __init__(self, detectors: Iterable[DetectionRule], patchers: Iterable[Patcher]):
        self.detectors = detectors
        self.patchers = patchers

    def search(self, src: str) -> List[CodeWarning]:
        out: List[CodeWarning] = []
        for d in self.detectors:
            out.extend(d.search(src))
        return sorted(out, key=lambda w: (w.lineno, w.col))

    def patch(self, src: str) -> str:
        tree = ast.parse(src)
        for p in self.patchers:
          tree = p.patch(tree)
        return ast.unparse(tree)

# None Compares

## Detector

In [18]:
class OsPathJoinDetectionRule(DetectionRule):
    name = "Detect os.path.join(...)"

    def search(self, src: str) -> List[CodeWarning]:
        # Patrón ajustado a os.path.join.
        tree = ast.parse(src)
        warnings: List[CodeWarning] = []
        for node in ast.walk(tree):
            if isinstance(node, ast.Call) and isinstance(node.func, ast.Attribute):
                f = node.func
                if f.attr == 'join' and isinstance(f.value, ast.Attribute) and f.value.attr == 'path' and isinstance(f.value.value, ast.Name) and f.value.value.id == 'os':
                    warnings.append(CodeWarning("Use pathlib.Path en lugar de os.path.join", node.lineno, node.col_offset))
        return warnings

## Patcher

In [19]:
class OsPathJoinPatcher(Patcher):
    name = "Fix os.path.join to Path division"

    def patch(self, tree: ast.AST) -> ast.AST:
        # Reemplaza join por Path()/operador '/'.
        class T(ast.NodeTransformer):
            def visit_Call(self, node):
                if (isinstance(node.func, ast.Attribute) and node.func.attr == 'join'
                    and isinstance(node.func.value, ast.Attribute) and node.func.value.attr == 'path'
                    and isinstance(node.func.value.value, ast.Name) and node.func.value.value.id == 'os'):
                    cur = ast.Call(func=ast.Name('Path', ast.Load()), args=[node.args[0]], keywords=[])
                    for arg in node.args[1:]:
                        cur = ast.BinOp(left=cur, op=ast.Div(), right=arg)
                    return cur
                return self.generic_visit(node)
        tree = T().visit(tree)
        tree.body.insert(0, ast.ImportFrom(module='pathlib', names=[ast.alias(name='Path')], level=0))
        return ast.fix_missing_locations(tree)

## Tests

In [20]:
import unittest

class TestOsPathJoinToPath(unittest.TestCase):
    def setUp(self):
        self.l = DCCLinter([OsPathJoinDetectionRule()], [OsPathJoinPatcher()])

    def test_detect(self):
        s = "import os\np = os.path.join('a','b')\n"
        self.assertEqual(len(self.l.search(s)), 1)

    def test_patch_two_args(self):
        s = "import os\np = os.path.join('a','b')\n"
        out = self.l.patch(s)
        self.assertIn("from pathlib import Path", out)
        self.assertIn("Path('a') / 'b'", out)

    def test_patch_many_args(self):
        s = "import os\np = os.path.join('a','b','c','d')\n"
        out = self.l.patch(s)
        self.assertIn("Path('a') / 'b' / 'c' / 'd'", out)

    # Tests extra
    def test_patch_variables(self):
        s = "import os\nbase='a'\nname='b'\np = os.path.join(base, name)\n"
        out = self.l.patch(s)
        self.assertIn("Path(base) / name", out)

    def test_no_warning_for_pathlib(self):
        s = "from pathlib import Path\np = Path('a') / 'b'\n"
        self.assertEqual(len(self.l.search(s)), 0)

    def test_three_args_mixed(self):
        s = "import os\nbase='root'\nsub='data'\np = os.path.join(base, sub, 'file.txt')\n"
        out = self.l.patch(s)
        self.assertIn("Path(base) / sub / 'file.txt'", out)

In [21]:
runTests(TestOsPathJoinToPath)

......
----------------------------------------------------------------------
Ran 6 tests in 0.005s

OK
.....
----------------------------------------------------------------------
Ran 6 tests in 0.005s

OK


## Real World Code

In [22]:
def find_cell(comment):
    """
    Busca la primera celda en el cuaderno actual cuya primera línea
    comience con el comentario dado (por ejemplo, "# Case Study 1").

    Parámetros:
        comment (str): El comentario que se quiere buscar.

    Retorna:
        str | None: El código fuente de la celda encontrada, o None si no existe.
    """
    for cell_source in In:  # `In` contiene el código fuente de todas las celdas ejecutadas
        lines = cell_source.strip().splitlines()
        if lines and lines[0].strip().startswith(comment):  # revisa la primera línea
            return cell_source
    return None

In [23]:
# Case Study 1
import os
base = ''
full = os.path.join(base, 'logs', '2025')

# Dentro de una función
def build_user_log(user):
    return os.path.join('/var', 'log', user)

# En un bucle
paths = []
for month in ['01','02','03']:
    paths.append(os.path.join('/data', '2025', month))

# Con variables y literales
root = '/tmp'
sub = 'cache'
leaf = 'index.json'
cache_file = os.path.join(root, sub, leaf)

In [24]:
linter = DCCLinter([OsPathJoinDetectionRule()], [OsPathJoinPatcher()])

bad_code =  find_cell("# Case Study 1")
warnings = linter.search(bad_code)
print("-= Warnings =-")
for w in warnings:
    print(w)
good_code = linter.patch(bad_code)
print("-= Fixed Code =-")
print(good_code)

-= Warnings =-
CodeWarning(message='Use pathlib.Path en lugar de os.path.join', lineno=4, col=7, end_lineno=None, end_col=None)
CodeWarning(message='Use pathlib.Path en lugar de os.path.join', lineno=9, col=11, end_lineno=None, end_col=None)
CodeWarning(message='Use pathlib.Path en lugar de os.path.join', lineno=14, col=17, end_lineno=None, end_col=None)
CodeWarning(message='Use pathlib.Path en lugar de os.path.join', lineno=20, col=13, end_lineno=None, end_col=None)
-= Fixed Code =-
from pathlib import Path
import os
base = ''
full = Path(base) / 'logs' / '2025'

def build_user_log(user):
    return Path('/var') / 'log' / user
paths = []
for month in ['01', '02', '03']:
    paths.append(Path('/data') / '2025' / month)
root = '/tmp'
sub = 'cache'
leaf = 'index.json'
cache_file = Path(root) / sub / leaf
